In [2]:


from deap import base
from deap import creator
from deap import gp
from deap import tools



pset = PrimitiveSet("MAIN", 2)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
pset.addTerminal(3)




pset.addPrimitive(operator.neg, 1)




expr = genFull(pset, min_=1, max_=3)
tree = PrimitiveTree(expr)

E:\Sofwares\Anaconda\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


NameError: name 'PrimitiveSet' is not defined

In [37]:
import random
import operator
import numpy as np
import pandas as pd
from binarytree import Node,build
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import operator

from deap import base
from deap import creator
from deap import gp
from deap import tools


pset = gp.PrimitiveSet("main", 9)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
a=4
pset.addTerminal(4)


pset.renameArguments(ARG0="thickness")
pset.renameArguments(ARG1="size")
pset.renameArguments(ARG2="shape")
pset.renameArguments(ARG3="ma")
pset.renameArguments(ARG4="se")
pset.renameArguments(ARG5="bn")
pset.renameArguments(ARG6="bc")
pset.renameArguments(ARG7="nn")
pset.renameArguments(ARG8="mitoses")


expr = gp.genFull(pset, min_=1, max_=5)
tree = gp.PrimitiveTree(expr)

creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=3, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)



toolbox.register("row", tools.initRepeat, list, toolbox.individual, n=10)
toolbox.register("population", tools.initRepeat, list, toolbox.row, n=10)



x=toolbox.population()
print(x[1][1])
print(x[9][9])

tree = gp.PrimitiveTree(expr)

function = gp.compile(str(x[1][1]), pset)


add(mul(mul(mul(max(se, bc), add(bn, shape)), add(mul(size, shape), max(4, 4))), add(mul(mul(bn, se), add(mitoses, nn)), mul(mul(thickness, shape), mul(ma, nn)))), max(max(add(add(thickness, se), add(thickness, se)), mul(max(bc, size), max(bc, thickness))), add(add(max(ma, 4), add(ma, 4)), add(add(4, shape), add(mitoses, bn)))))
add(add(max(mul(mul(shape, mitoses), max(shape, bc)), add(mul(nn, se), add(4, se))), add(mul(max(bc, shape), add(ma, 4)), max(add(mitoses, thickness), add(shape, thickness)))), mul(add(max(mul(se, bc), max(thickness, nn)), add(add(size, bn), add(bc, se))), mul(add(mul(bc, 4), mul(se, shape)), mul(add(ma, shape), mul(ma, mitoses)))))


E:\Sofwares\Anaconda\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
E:\Sofwares\Anaconda\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [48]:
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn import preprocessing

data = pd.read_csv('Dataset_MLAI.txt',sep = ',',header = None)
#display(data)
data[6] =  pd.to_numeric(data[6], errors='coerce') 
# drop nas
data = data.dropna()

x_features = data.iloc[:,1:10]
y_features = data.iloc[:,10]
y_list = list(y_features.values.T.ravel())
x_array = x_features.values

scaler = preprocessing.StandardScaler()
scaled_X = scaler.fit_transform(x_features)


x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=42,shuffle = True)

list42=[]
for i in range(30):
    if(function(scaled_X[i,0],scaled_X[i,1],scaled_X[i,2],scaled_X[i,3],scaled_X[i,4],scaled_X[i,5],scaled_X[i,6],scaled_X[i,7],scaled_X[i,8])<0):
        list42.append(4)
    else:
        list42.append(2)
print("Batch 1 : ")
print(list42)
print("\n")

for j in range(19):
    x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
    listRand=[]
    scaled_X = scaler.fit_transform(x_train)
    
    for i in range(30):
        if(function(scaled_X[i,0],scaled_X[i,1],scaled_X[i,2],scaled_X[i,3],scaled_X[i,4],scaled_X[i,5],scaled_X[i,6],scaled_X[i,7],scaled_X[i,8])<0):
            listRand.append(4)
        else:
            listRand.append(2)
    print("Batch"+str(j+2)+" : ")
    print(listRand)
    print("\n")
# print(x_train)


Batch 1 : 
[2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Batch2 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Batch3 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2]


Batch4 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2]


Batch5 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Batch6 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4]


Batch7 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Batch8 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Batch9 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Batch10 : 
[2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 